In [4]:
using Distributed
# addprocs(4)

@everywhere begin
    using Revise
    using MendelIHT
    using SnpArrays
    using Random
    using GLM
    using DelimitedFiles
    using Test
    using Distributions
    using LinearAlgebra
    using CSV
    using DataFrames
end

# First simulate multivariate Gaussian traits

In [2]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
r = 2     # number of traits

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray("multivariate_$(r)traits.bed", n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n) 

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_Σ, true_b, correct_position = simulate_random_response(xla, k, r);

In [3]:
# save true SNP's position and effect size
open("multivariate_$(r)traits_true_beta.txt", "w") do io
    println(io, "snpID,effectsize")
    for pos in correct_position
        println(io, "snp$pos,", true_b[pos])
    end
end

# create `.bim` and `.bam` files using phenotype
make_bim_fam_files(x, Y, "multivariate_$(r)traits")

# Run IHT with simple gradient update

In [3]:
Yt = Matrix(Y')
Zt = Matrix(z')
@time result = fit_iht(Yt, Transpose(xla), Zt, k=10, verbose=false)

v.df = [0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0; 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0]
v.B = [1.5516236912422103, -0.3696378308110806, 0.7359394140835553, 0.6639448366269044, -1.3474596982982028, 0.29715970864320057, -0.2849927601385842, -0.8331017779743192, 0.3248807976411838, 0.3363355150124857, 0.3646010384703131, 0.2964188052006387, -1.1379417000342935, -0.788453230705004, 0.3680544381781284, -0.4909628161857173, -1.2439619095083274, 0.453677248752949, 0.34161067266004913, 1.2899176314533622]
v.df = [-2.803652821032988 -5.273576914615036 1.0346083479906945 9.033913850714168 -11.792983487882186 -29.28703824569754 31.91190710105664 5.396692118218743 0.1562089302047034 8.891696162379223 15.912316064075156 -23.21532913524465 0.7219812654881481 -2.0362176967145666 -6.4540948077263485 6.31307242481383 11.746867632305976 -6.0435208765584445 10.393670949063106 -1.0147330640905368; 12.526992583706887 -1.45491


Compute time (sec):     1.7867960929870605
Final loglikelihood:    2635.989391876837
Iterations:             14

Trait 1: IHT estimated 10 nonzero SNP predictors
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782   -0.387253
   2 │      901    0.776228
   3 │     1204    0.73234
   4 │     1306   -1.45682
   5 │     1427    0.0570532
   6 │     1655   -0.186513
   7 │     3160   -0.845163
   8 │     4201    0.309019
   9 │     6047    0.152959
  10 │     6879   -1.23302

Trait 1: IHT estimated 0 non-genetic predictors
0×2 DataFrame

Trait 2: IHT estimated 10 nonzero SNP predictors
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │       46    1.73289
   2 │     4797   -1.20165
   3 │     5616   -0.962282
   4 │     6072    0.536906
   5 │     6573   -0.591373
   6 │     7236    0.509888
   7 │     7407   -0.0718126
   8 │     7667    0.340733
   9 │     90

# Run IHT with full gradient update

In [4]:
Yt = Matrix(Y')
Zt = Matrix(z')
@time result = fit_iht(Yt, Transpose(xla), Zt, k=10, fullIHT=true, max_step=10, verbose=false)

v.df = [38.46034346894085 -409.15811579798645 814.6232850332892 734.9313184529236 -1491.5250153364145 328.93090572667063 -922.1738830031752 359.61582917564544 -5.025553558897185 328.11078767792634 -85.31289303492862 99.50197614747124 -152.7778451364754 -19.04322552429309 -1376.961632693455 -27.660700649654157 16.57419092139826 65.84688860875383; 1717.517453619762 -2.875396761791457 -18.41862072980716 -55.66765524131222 18.351630126237367 309.9203577215312 -23.79893293490111 143.92072647258416 403.5828085863514 -58.29650106764749 -1259.6061416450448 -872.7516811853986 407.40543278703603 -543.4547117556547 -98.83605257110203 502.18271185300586 378.134399434633 1427.8307673810698]
v.B = [5.277026895919724, -1.257127476171674, 2.502908471819727, 2.258057014766396, -4.582671113635067, 1.0106314976657913, -2.8333548361132603, 1.104910112417689, 1.2399974923995691, 1.0081117066779723, -3.8701065154928744, -2.6815064297414213, 1.2517424040320493, -1.669750211041403, -4.230678154133951, 1.54294


Compute time (sec):     0.0
Final loglikelihood:    99468.96790624551
Iterations:             0

Trait 1: IHT estimated 10 nonzero SNP predictors
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782    -1.1156
   2 │      901     2.1656
   3 │     1204     1.99475
   4 │     1306    -4.0089
   5 │     1993     0.752676
   6 │     2127     0.11654
   7 │     3160    -2.43156
   8 │     4201     0.966386
   9 │     4705     0.770299
  10 │     6879    -3.62476

Trait 1: IHT estimated 0 non-genetic predictors
0×2 DataFrame

Trait 2: IHT estimated 10 nonzero SNP predictors
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │        4     0.109934
   2 │       46     4.73693
   3 │     4656     0.960335
   4 │     4797    -3.46211
   5 │     5616    -2.45509
   6 │     6072     1.17425
   7 │     6573    -1.50826
   8 │     7236     1.37251
   9 │     9626     0.89

## Check answer

In [107]:
# first beta
β1 = result.beta[1, :]
true_b1_idx = findall(!iszero, true_b[:, 1])
[β1[true_b1_idx] true_b[true_b1_idx, 1]]

10×2 Array{Float64,2}:
 -1.1156    -0.402269
  2.1656     0.758756
  1.99475    0.729135
 -4.0089    -1.47163
  0.0       -0.172668
 -2.43156   -0.847906
  0.966386   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -3.62476   -1.24972

In [108]:
# second beta
β2 = result.beta[2, :]
true_b2_idx = findall(!iszero, true_b[:, 2])
[β2[true_b2_idx] true_b[true_b2_idx, 2]]

10×2 Array{Float64,2}:
  4.73693   1.73729
 -3.46211  -1.19911
  0.0       0.0121193
 -2.45509  -0.969569
  1.17425   0.540525
 -1.50826  -0.609556
  1.37251   0.481189
  0.0      -0.0524866
  0.0       0.31182
  3.85105   1.29813

In [109]:
# covariance matrix
[vec(result.Σ) vec(true_Σ)]

4×2 Array{Float64,2}:
 0.463147   1.22512
 0.0358997  1.23674
 0.0358997  1.23674
 0.634873   1.494

# Test Cross validation

In [9]:
Random.seed!(2020)
Yt = Matrix(Y')
Zt = Matrix(z')
@time cv_iht(Yt, x, Zt);



Crossvalidation Results:
	k	MSE
	1	2506.1361026992413
	2	1707.4609062019433
	3	1297.253354519584
	4	985.2280061442002
	5	819.8080675324194
	6	739.6359401670024
	7	641.5504839534303
	8	612.7042220588955
	9	608.3277089584982
	10	610.9984342539259
	11	614.807751686425
	12	616.615453582629
	13	621.4468351156702
	14	628.0553031281725
	15	628.5181881547544
	16	626.4682026915706
	17	636.4936893268582
	18	635.9073694771134
	19	630.7374247665347
	20	633.3757710953382
284.723952 seconds (151.20 k allocations: 823.455 MiB, 0.05% gc time)


# Compare with univariate IHT

+ `fit_iht`: ~20 times slower
+ `cv_iht`: ~70 times slower

In [40]:
n = 1000  # number of samples
p = 10000 # number of SNPs
k = 10    # number of causal SNPs per trait
d = Normal
l = IdentityLink()

# set random seed for reproducibility
Random.seed!(2021)

# simulate `.bed` file with no missing data
x = simulate_random_snparray(undef, n, p)
xla = SnpLinAlg{Float64}(x, model=ADDITIVE_MODEL, center=true, scale=true) 

# intercept is the only nongenetic covariate
z = ones(n) 

# simulate response y, true model b, and the correct non-0 positions of b
Y, true_b, correct_position = simulate_random_response(xla, k, d, l);

In [42]:
@time result = fit_iht(Y, xla, z, k=10)

****                   MendelIHT Version 1.4.0                  ****
****     Benjamin Chu, Kevin Keys, Chris German, Hua Zhou       ****
****   Jin Zhou, Eric Sobel, Janet Sinsheimer, Kenneth Lange    ****
****                                                            ****
****                 Please cite our paper!                     ****
****         https://doi.org/10.1093/gigascience/giaa044        ****

Running sparse linear regression
Link functin = IdentityLink()
Sparsity parameter (k) = 10
Prior weight scaling = off
Doubly sparse projection = off
Debias = off

Converging when tol < 0.0001:
Iteration 1: loglikelihood = -1568.8072402506227, tol = 1.2369352935298255
Iteration 2: loglikelihood = -1485.2388920745977, tol = 0.1246509194436555
Iteration 3: loglikelihood = -1473.4688017685708, tol = 0.060408468355759436
Iteration 4: loglikelihood = -1472.7990029487871, tol = 0.0051541599786159185
Iteration 5: loglikelihood = -1472.6215787426543, tol = 0.0027074009598392432
Iteration


IHT estimated 10 nonzero SNP predictors and 0 non-genetic predictors.

Compute time (sec):     0.06568217277526855
Final loglikelihood:    -1472.5590426182382
Iterations:             10

Selected genetic predictors:
10×2 DataFrame
 Row │ Position  Estimated_β 
     │ Int64     Float64     
─────┼───────────────────────
   1 │      782    -0.438598
   2 │      901     0.747536
   3 │     1204     0.691351
   4 │     1306    -1.42482
   5 │     1655    -0.194714
   6 │     3160    -0.86084
   7 │     3936    -0.147349
   8 │     4201     0.338177
   9 │     4402    -0.126286
  10 │     6879    -1.21081

Selected nongenetic predictors:
0×2 DataFrame

In [43]:
# compare beta values
[result.beta[correct_position] true_b[correct_position]]

10×2 Array{Float64,2}:
 -0.438598  -0.402269
  0.747536   0.758756
  0.691351   0.729135
 -1.42482   -1.47163
 -0.194714  -0.172668
 -0.86084   -0.847906
  0.338177   0.296183
  0.0       -0.0034339
  0.0        0.125965
 -1.21081   -1.24972

### Cross validation timing

In [8]:
# 1 cores
Random.seed!(2020)
@time cv_iht(Y, x, z);



Crossvalidation Results:
	k	MSE
	1	1431.385325867502
	2	1431.569108502913
	3	1429.2348660984221
	4	1429.5730864011514
	5	1429.4077307081652
	6	1429.2323485244572
	7	1429.5277693297523
	8	1429.4669590879946
	9	1429.4376255311238
	10	1429.526751070809
	11	1429.5205225689476
	12	1429.6007792570374
	13	1430.324407382292
	14	1430.2241106412075
	15	1430.2935769756937
	16	1430.0160991249024
	17	1430.2798769215433
	18	1430.7721172663867
	19	1430.8725535864496
	20	1429.7969640813615
  4.344964 seconds (10.10 M allocations: 343.536 MiB, 0.70% gc time)


In [7]:
# 4 cores
Random.seed!(2020)
@time cv_iht(Y, x, z);



Crossvalidation Results:
	k	MSE
	1	1431.385325867502
	2	1431.569108502913
	3	1429.2348660984221
	4	1429.5730864011514
	5	1429.4077307081652
	6	1429.2323485244572
	7	1429.5277693297523
	8	1429.4669590879946
	9	1429.4376255311238
	10	1429.526751070809
	11	1429.5205225689476
	12	1429.6007792570374
	13	1430.324407382292
	14	1430.2241106412075
	15	1430.2935769756937
	16	1430.0160991249024
	17	1430.2798769215433
	18	1430.7721172663867
	19	1430.8725535864496
	20	1429.7969640813615
  4.223226 seconds (10.10 M allocations: 343.629 MiB, 0.88% gc time)


# GEMMA multivariate results

In [37]:
gemma_df = CSV.read("gemma.result.assoc.txt", DataFrame)

# pvalues
pval_wald = gemma_df[!, :p_wald]
pval_lrt = gemma_df[!, :p_lrt]
pval_score = gemma_df[!, :p_score]

# estimated beta
estim_β1 = gemma_df[!, :beta_1]
estim_β2 = gemma_df[!, :beta_2]

# estimated covariance matrix
estim_σ11 = gemma_df[!, :Vbeta_1_1]
estim_σ12 = gemma_df[!, :Vbeta_1_2]
estim_σ22 = gemma_df[!, :Vbeta_2_2];

In [39]:
correct_snps = [x[1] for x in correct_position]  # truely causal snps
signif_snps = findall(x -> x ≤ 0.05/p, pval_lrt) # gemma's selected snps
signif_snps ∩ correct_snps

12-element Array{Int64,1}:
   46
  901
 1204
 1306
 3160
 4797
 5616
 6072
 6573
 6879
 7236
 9763